In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import string
import nltk
from nltk.corpus import stopwords
from typing import List, Optional
import re
import kagglehub

c:\Users\Derick Sayavong\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LinkedIn Job Postings

In [4]:
combine-notebooks-and-test-unlabelled-datapath = os.path.join("data", "linkedin_postings.csv")
linkedin_job_df = pd.read_csv(path)
linkedin_job_df.head(10)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0
5,91700727,Downtown Raleigh Alliance,Economic Development and Planning Intern,Job summary:The Economic Development & Plannin...,20.0,HOURLY,"Raleigh, NC",1481176.0,9.0,NaN,...,NaN,1.713456e+12,NaN,0,INTERNSHIP,USD,BASE_SALARY,35360.0,27601.0,37183.0
6,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,YEARLY,United States,81942316.0,7.0,NaN,...,NaN,1.712861e+12,NaN,0,CONTRACT,USD,BASE_SALARY,180000.0,NaN,NaN
7,112576855,NaN,Building Engineer,Summary: Due to the pending retirement of our ...,120000.0,YEARLY,"San Francisco, CA",NaN,2.0,NaN,...,NaN,1.712443e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,105000.0,94101.0,6075.0
8,1218575,Children's Nebraska,Respiratory Therapist,"At Children’s, the region’s only full-service ...",NaN,NaN,"Omaha, NE",721189.0,3.0,NaN,...,• Requires the ability to communicate effectiv...,1.712348e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,68102.0,31055.0
9,2264355,Bay West Church,Worship Leader,It is an exciting time to be a part of our chu...,NaN,MONTHLY,"Palm Bay, FL",28631247.0,5.0,350.0,...,"Knowledge, Skills and Abilities: 1. Proficient...",1.712456e+12,NaN,0,PART_TIME,USD,BASE_SALARY,4200.0,32905.0,12009.0


In [5]:
print(f'Number of rows: {len(linkedin_job_df)}')

Number of rows: 123849


Drop irrelvant columns

In [6]:
linkedin_job_df_clean = linkedin_job_df.drop(columns = ["fips", "zip_code", "listed_time", "job_id", "company_id", "views", 
                                "closed_time", "med_salary", "applies", "application_type", 
                                "original_listed_time", "job_posting_url", "application_url", "expiry", 
                                "compensation_type", "normalized_salary", "posting_domain", "remote_allowed", 
                                "formatted_experience_level", "skills_desc", "sponsored", "work_type", "formatted_work_type"])

In [7]:
linkedin_job_df_clean.isna().sum()

company_name     1719
title               0
description         7
max_salary      94056
pay_period      87776
location            0
min_salary      94056
currency        87776
dtype: int64

Drop samples with no job descriptions

In [8]:
linkedin_job_df_clean = linkedin_job_df_clean.dropna(subset=["description"])

In [9]:
linkedin_job_df_clean.isna().sum()

company_name     1718
title               0
description         0
max_salary      94050
pay_period      87770
location            0
min_salary      94050
currency        87770
dtype: int64

In [10]:
linkedin_job_df_clean.head()

,company_name,title,description,max_salary,pay_period,location,min_salary,currency
0,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",17.0,USD
1,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",30.0,USD
2,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",45000.0,USD
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",140000.0,USD
4,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",60000.0,USD


# Indeed Job Postings

In [11]:
path = os.path.join("data", "indeed-job-listings-information.csv")
indeed_job_df = pd.read_csv(path)
indeed_job_df.head(10)

,jobid,company_name,date_posted_parsed,job_title,description_text,benefits,job_type,location,salary_formatted,company_rating,company_reviews_count,country,date_posted,description,company_link,domain,apply_link,url,is_expired,timestamp
0,d46480488e3aa20c,FitLab INC,"""2024-08-04T14:55:36.532Z""",FitLab Email Marketing Manager,Description: \n Who Is FitLab: FitLab is a p...,"[""401(k)"",""Dental insurance"",""Disability insur...",Full-time,California,"$90,000 - $105,000 a year",4.5,4,US,Just posted,Description: \n Who Is FitLab: FitLab is a ...,https://www.indeed.com/cmp/Fitlab?campaignid=m...,https://indeed.com,https://www.indeed.com/viewjob?jk=d46480488e3a...,https://www.indeed.com/viewjob?jk=d46480488e3a...,False,2024-08-04 00:00:00.000
1,ba9bf4e04cfd9cdb,Buccal Up Dental,"""2024-08-04T12:29:09.402Z""",Customer Service Representative - Work From Home,The Customer Associate is the first point of c...,"[""401(k) matching"",""Continuing education credi...",Permanent,"Sugar Land, TX 77479",$28 - $50 an hour,5.0,4,US,Just posted,The Customer Associate is the first point of ...,https://www.indeed.com/cmp/Buccal-Up-Dental-Gr...,https://indeed.com,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,False,2024-08-04 00:00:00.000
2,9e4031d06dd42d32,Peraton,"""2024-08-01T19:30:03.000Z""",Information Technology Support Specialist,About Peraton Peraton is a next-generation na...,"[""Dental insurance"",""Disability insurance"",""He...",Full-time,United States,"$66,000 - $106,000 a year",3.1,297,US,2 days ago,About Peraton Peraton is a next-generation n...,https://www.indeed.com/cmp/Peraton?campaignid=...,https://indeed.com,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,False,2024-08-04 00:00:00.000
3,46554b6bae3dfc9f,Samu social de Paris,"""2022-04-27T20:55:01.000Z""",Opérateur.trice de réservation hotelière (de j...,"Dans ce cadre, vous êtes amené/e à réaliser le...","[""Prise en charge du transport quotidien"",""RTT""]",CDD,75012 Paris,"1 964,03 € par mois",3.6,15,FR,il y a 30+ jours,"Dans ce cadre, vous êtes amené/e à réaliser...",https://fr.indeed.com/cmp/Samusocial-De-Paris?...,https://fr.indeed.com,https://fr.indeed.com/viewjob?jk=46554b6bae3dfc9f,https://fr.indeed.com/viewjob?jk=46554b6bae3dfc9f,False,2024-08-17 00:00:00.000
4,06e52e564455d499,Terideal,"""2024-08-14T19:05:19.000Z""",Chauffeur Opérateur Curage H/F,Informations générales ...,"[""Intéressement et participation""]","CDI, Temps plein",Wissous (91),De 24 000 € à 28 000 € par an,2.6,16,FR,il y a 3 jours,...,https://fr.indeed.com/cmp/Terideal?campaignid=...,https://fr.indeed.com,https://fr.indeed.com/viewjob?jk=06e52e564455d499,https://fr.indeed.com/viewjob?jk=06e52e564455d499,False,2024-08-17 00:00:00.000
5,2f5383b3496e2158,Samu social de Paris,"""2022-04-27T20:55:02.000Z""",Opérateur.trice de réservation hôtelière (trav...,"Dans ce cadre, vous êtes amené/e à réaliser le...","[""Prise en charge du transport quotidien"",""RTT""]",CDD,75012 Paris,"1 964,03 € par mois",3.6,15,FR,il y a 30+ jours,"Dans ce cadre, vous êtes amené/e à réaliser...",https://fr.indeed.com/cmp/Samusocial-De-Paris?...,https://fr.indeed.com,https://fr.indeed.com/viewjob?jk=2f5383b3496e2158,https://fr.indeed.com/viewjob?jk=2f5383b3496e2158,False,2024-08-17 00:00:00.000
6,449318c8b6d4b414,Marine Nationale,"""2023-09-19T10:26:42.000Z""",Opérateur linguistique d'interception sous sta...,Poste opérateur linguistique d'interception so...,"[""Prise en charge du transport quotidien"",""Réd...","Temps plein, CDD",Paris (75),De 1 600 € à 2 300 € par mois,3.9,551,FR,il y a 30+ jours,Poste opérateur linguistique d'interception so...,https://fr.indeed.com/cmp/Marine-Nationale?cam...,https://fr.indeed.com,https://fr.indeed.com/viewjob?jk=449318c8b6d4b414,https://fr.indeed.com/viewjob?jk=449318c8b6d4b414,False,2024-08-17 00:00:00.000
7,4fe58dcd36ddb4b3,Cramif,"""2024-06-14T10:33:53.000Z""",Opérateur de saisie H/F,Rejo

In [12]:
print(f'Number of rows: {len(indeed_job_df)}')

Number of rows: 1000


Drop Irrelevant Columns

In [13]:
indeed_job_df_clean = indeed_job_df.drop(columns = ["date_posted_parsed", "jobid", "job_type", "company_rating", "company_reviews_count", "date_posted", 
                                                    "company_link", "domain", "apply_link",
                                                      "url", "is_expired", "timestamp", "description_text"])

Only look at US postings

In [14]:
indeed_job_df_clean = indeed_job_df_clean[indeed_job_df_clean["country"] == "US"]
indeed_job_df_clean = indeed_job_df_clean.reset_index(drop=True)
indeed_job_df_clean.head(10)

,company_name,job_title,benefits,location,salary_formatted,country,description
0,FitLab INC,FitLab Email Marketing Manager,"[""401(k)"",""Dental insurance"",""Disability insur...",California,"$90,000 - $105,000 a year",US,Description: \n Who Is FitLab: FitLab is a ...
1,Buccal Up Dental,Customer Service Representative - Work From Home,"[""401(k) matching"",""Continuing education credi...","Sugar Land, TX 77479",$28 - $50 an hour,US,The Customer Associate is the first point of ...
2,Peraton,Information Technology Support Specialist,"[""Dental insurance"",""Disability insurance"",""He...",United States,"$66,000 - $106,000 a year",US,About Peraton Peraton is a next-generation n...
3,American College of Healthcare Sciences,Student Accounts Specialist,"[""401(k) matching"",""Dental insurance"",""Disabil...",Remote,$25 - $27 an hour,US,About ACHS \n American College of Healthcare...
4,Torrid,Integrated Marketing Coordinator,"[""401(k) matching"",""Childcare"",""Dental insuran...",Remote,$21.51 - $26.31 an hour,US,At Torrid we're committed to cultivating a we...
5,Olympus Corporation of the Americas,Global Manager of Medical Content - Remote,"[""401(k) matching"",""Adoption assistance"",""Dent...","Boston, MA 02108","$99,267 - $138,973 a year",US,Workplace Flexibility: Field For more than ...
6,NAVIENT,Healthcare Receivables Specialist - September ...,"[""401(k) matching"",""Dental insurance"",""Health ...",Remote,$16.50 an hour,US,About Xtend Healthcare Xtend Healthcare ...
7,Trinity Underground,Payroll Clerk / Remote,"[""401(k)"",""Dental insurance"",""Health insurance...","Lawrenceville, GA 30044","$48,000 - $85,000 a year",US,Description We are looking for a qualifi...
8,Neuberger Berman,Private Markets Fundraising Analyst/Associate,"[""401(k)"",""Dental insurance"",""Health insurance...","New York, NY","$100,000 - $140,000 a year",US,Summary:\n Neuberger Berman is an empl...
9,Trinity Underground,Payroll Specialist / Work From Home,"[""401(k)"",""Dental insurance"",""Health insurance...","Suwanee, GA 30024","$48,000 - $85,000 a year",US,Description We are looking for a qualifi...


See how many US job postings there are

In [15]:
print(f'Number of US job postings: {len(indeed_job_df_clean)}')

Number of US job postings: 968


In [16]:
indeed_job_df_clean.isna().sum()

company_name        0
job_title           0
benefits            0
location            0
salary_formatted    0
country             0
description         0
dtype: int64

# Preprocessing

In [17]:
def preprocess_remove_sw_punc(text):
	text = text.lower()  # convert to lowercase
	text = ''.join([char for char in text if not char.isdigit()]) # remove numerical characters
	text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove URLs
	text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

	stop_words = set(stopwords.words('english'))

	text = ' '.join([word for word in text.split() if word not in stop_words]) # remove stop words

	text = " ".join(text.split()) # remove extra spaces


	return text

In [18]:
# w/out removing stop words and punctuation
def preprocess(text):
	text = text.lower()  # convert to lowercase
	text = ''.join([char for char in text if not char.isdigit()]) # remove numerical characters
	text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove URLs
	text = " ".join(text.split()) # remove extra spaces

	return text

In [19]:
indeed_job_df_clean_no_sw = indeed_job_df_clean.copy()
indeed_job_df_clean_no_sw["description"] = indeed_job_df_clean["description"].apply(preprocess_remove_sw_punc)
indeed_job_df_clean_no_sw.head(10)


,company_name,job_title,benefits,location,salary_formatted,country,description
0,FitLab INC,FitLab Email Marketing Manager,"[""401(k)"",""Dental insurance"",""Disability insur...",California,"$90,000 - $105,000 a year",US,description fitlab fitlab performance lifestyl...
1,Buccal Up Dental,Customer Service Representative - Work From Home,"[""401(k) matching"",""Continuing education credi...","Sugar Land, TX 77479",$28 - $50 an hour,US,customer associate first point contact address...
2,Peraton,Information Technology Support Specialist,"[""Dental insurance"",""Disability insurance"",""He...",United States,"$66,000 - $106,000 a year",US,peraton peraton next generation national secur...
3,American College of Healthcare Sciences,Student Accounts Specialist,"[""401(k) matching"",""Dental insurance"",""Disabil...",Remote,$25 - $27 an hour,US,achs american college healthcare sciences achs...
4,Torrid,Integrated Marketing Coordinator,"[""401(k) matching"",""Childcare"",""Dental insuran...",Remote,$21.51 - $26.31 an hour,US,torrid committed cultivating welcoming inclusi...
5,Olympus Corporation of the Americas,Global Manager of Medical Content - Remote,"[""401(k) matching"",""Adoption assistance"",""Dent...","Boston, MA 02108","$99,267 - $138,973 a year",US,workplace flexibility field years olympus focu...
6,NAVIENT,Healthcare Receivables Specialist - September ...,"[""401(k) matching"",""Dental insurance"",""Health ...",Remote,$16.50 an hour,US,xtend healthcare xtend healthcare revenue cycl...
7,Trinity Underground,Payroll Clerk / Remote,"[""401(k)"",""Dental insurance"",""Health insurance...","Lawrenceville, GA 30044","$48,000 - $85,000 a year",US,description looking qualified payroll clerk as...
8,Neuberger Berman,Private Markets Fundraising Analyst/Associate,"[""401(k)"",""Dental insurance"",""Health insurance...","New York, NY","$100,000 - $140,000 a year",US,summary neuberger berman employee owned global...
9,Trinity Underground,Payroll Specialist / Work From Home,"[""401(k)"",""Dental insurance"",""Health insurance...","Suwanee, GA 30024","$48,000 - $85,000 a year",US,description looking qualified payroll clerk as...


In [20]:
linkedin_job_df_clean_no_sw = linkedin_job_df_clean.copy()
linkedin_job_df_clean_no_sw["description"] = linkedin_job_df_clean["description"].apply(preprocess_remove_sw_punc)
linkedin_job_df_clean_no_sw.head(10)

,company_name,title,description,max_salary,pay_period,location,min_salary,currency
0,Corcoran Sawyer Smith,Marketing Coordinator,job descriptiona leading real estate firm new ...,20.0,HOURLY,"Princeton, NJ",17.0,USD
1,NaN,Mental Health Therapist/Counselor,aspen therapy wellness committed serving clien...,50.0,HOURLY,"Fort Collins, CO",30.0,USD
2,The National Exemplar,Assitant Restaurant Manager,national exemplar accepting applications assis...,65000.0,YEARLY,"Cincinnati, OH",45000.0,USD
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,senior associate attorney elder law trusts est...,175000.0,YEARLY,"New Hyde Park, NY",140000.0,USD
4,NaN,Service Technician,looking hvac service tech experience commerica...,80000.0,YEARLY,"Burlington, IA",60000.0,USD
5,Downtown Raleigh Alliance,Economic Development and Planning Intern,job summary economic development planning inte...,20.0,HOURLY,"Raleigh, NC",14.0,USD
6,Raw Cereal,Producer,company descriptionraw cereal creative design ...,300000.0,YEARLY,United States,60000.0,USD
7,NaN,Building Engineer,summary due pending retirement building engine...,120000.0,YEARLY,"San Francisco, CA",90000.0,USD
8,Children's Nebraska,Respiratory Therapist,children’s region’s full service pediatric hea...,NaN,NaN,"Omaha, NE",NaN,NaN
9,Bay West Church,Worship Leader,exciting time part church looking right energe...,NaN,MONTHLY,"Palm Bay, FL",NaN,USD


In [21]:
indeed_job_df_clean_sw = indeed_job_df_clean.copy()
indeed_job_df_clean_sw["description"] = indeed_job_df_clean["description"].apply(preprocess)
indeed_job_df_clean_sw.head(10)

,company_name,job_title,benefits,location,salary_formatted,country,description
0,FitLab INC,FitLab Email Marketing Manager,"[""401(k)"",""Dental insurance"",""Disability insur...",California,"$90,000 - $105,000 a year",US,description: who is fitlab: fitlab is a perfor...
1,Buccal Up Dental,Customer Service Representative - Work From Home,"[""401(k) matching"",""Continuing education credi...","Sugar Land, TX 77479",$28 - $50 an hour,US,the customer associate is the first point of c...
2,Peraton,Information Technology Support Specialist,"[""Dental insurance"",""Disability insurance"",""He...",United States,"$66,000 - $106,000 a year",US,about peraton peraton is a next-generation nat...
3,American College of Healthcare Sciences,Student Accounts Specialist,"[""401(k) matching"",""Dental insurance"",""Disabil...",Remote,$25 - $27 an hour,US,about achs american college of healthcare scie...
4,Torrid,Integrated Marketing Coordinator,"[""401(k) matching"",""Childcare"",""Dental insuran...",Remote,$21.51 - $26.31 an hour,US,at torrid we're committed to cultivating a wel...
5,Olympus Corporation of the Americas,Global Manager of Medical Content - Remote,"[""401(k) matching"",""Adoption assistance"",""Dent...","Boston, MA 02108","$99,267 - $138,973 a year",US,workplace flexibility: field for more than yea...
6,NAVIENT,Healthcare Receivables Specialist - September ...,"[""401(k) matching"",""Dental insurance"",""Health ...",Remote,$16.50 an hour,US,about xtend healthcare xtend healthcare is a r...
7,Trinity Underground,Payroll Clerk / Remote,"[""401(k)"",""Dental insurance"",""Health insurance...","Lawrenceville, GA 30044","$48,000 - $85,000 a year",US,description we are looking for a qualified pay...
8,Neuberger Berman,Private Markets Fundraising Analyst/Associate,"[""401(k)"",""Dental insurance"",""Health insurance...","New York, NY","$100,000 - $140,000 a year",US,summary: neuberger berman is an employee-owned...
9,Trinity Underground,Payroll Specialist / Work From Home,"[""401(k)"",""Dental insurance"",""Health insurance...","Suwanee, GA 30024","$48,000 - $85,000 a year",US,description we are looking for a qualified pay...


In [22]:
linkedin_job_df_clean_sw = linkedin_job_df_clean.copy()
linkedin_job_df_clean_sw["description"] = linkedin_job_df_clean["description"].apply(preprocess)
linkedin_job_df_clean_sw.head(10)

,company_name,title,description,max_salary,pay_period,location,min_salary,currency
0,Corcoran Sawyer Smith,Marketing Coordinator,job descriptiona leading real estate firm in n...,20.0,HOURLY,"Princeton, NJ",17.0,USD
1,NaN,Mental Health Therapist/Counselor,"at aspen therapy and wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",30.0,USD
2,The National Exemplar,Assitant Restaurant Manager,the national exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",45000.0,USD
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,senior associate attorney - elder law / trusts...,175000.0,YEARLY,"New Hyde Park, NY",140000.0,USD
4,NaN,Service Technician,looking for hvac service tech with experience ...,80000.0,YEARLY,"Burlington, IA",60000.0,USD
5,Downtown Raleigh Alliance,Economic Development and Planning Intern,job summary:the economic development & plannin...,20.0,HOURLY,"Raleigh, NC",14.0,USD
6,Raw Cereal,Producer,company descriptionraw cereal is a creative de...,300000.0,YEARLY,United States,60000.0,USD
7,NaN,Building Engineer,summary: due to the pending retirement of our ...,120000.0,YEARLY,"San Francisco, CA",90000.0,USD
8,Children's Nebraska,Respiratory Therapist,"at children’s, the region’s only full-service ...",NaN,NaN,"Omaha, NE",NaN,NaN
9,Bay West Church,Worship Leader,it is an exciting time to be a part of our chu...,NaN,MONTHLY,"Palm Bay, FL",NaN,USD


Drop Duplicate Descriptions

In [23]:
num_rows = len(linkedin_job_df_clean_sw)
linkedin_job_df_clean_sw = linkedin_job_df_clean_sw.drop_duplicates(subset=["description"])
duplicates = num_rows - len(linkedin_job_df_clean_sw)

print(f'{duplicates} duplicates have been removed.')

19694 duplicates have been removed.


In [24]:
num_duplicates = linkedin_job_df_clean_sw["description"].duplicated().sum()
print(f"Number of duplicate linkedIn descriptions: {num_duplicates}") # Just to be sure there are no duplicates

Number of duplicate linkedIn descriptions: 0


In [25]:
num_rows = len(indeed_job_df_clean_sw)
indeed_job_df_clean_sw = indeed_job_df_clean_sw.drop_duplicates(subset=["description"])
duplicates = num_rows - len(indeed_job_df_clean_sw)

print(f'{duplicates} have been removed.')

29 have been removed.


In [26]:
num_duplicates = indeed_job_df_clean_sw["description"].duplicated().sum()
print(f"Number of duplicate Indeed descriptions: {num_duplicates}") # Just to be sure there are no duplicates

Number of duplicate Indeed descriptions: 0


In [27]:
print(f'Number of LinkedIn Rows: {len(linkedin_job_df_clean_sw)}')
print(f'Number of Indeed Rows: {len(indeed_job_df_clean_sw)}')

Number of LinkedIn Rows: 104148
Number of Indeed Rows: 939


Tokenize the data in the same way as the training and validation set

In [28]:
from transformers import BertModel, BertTokenizer

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [30]:
def tokenize_data(text, tokenizer, max_length=256):
	return tokenizer(
		text.tolist(),
		add_special_tokens=True,
		padding=True,
		truncation=True,
		max_length=max_length,
		return_tensors='pt'
	)

In [31]:
linkedin_encodings = tokenize_data(linkedin_job_df_clean_sw["description"], tokenizer)

In [32]:
print(linkedin_encodings)

{'input_ids': tensor([[  101,  3105,  6412,  ...,  1004, 10639,   102],
        [  101,  2012, 18567,  ...,  1007,  3510,   102],
        [  101,  1996,  2120,  ...,     0,     0,     0],
        ...,
        [  101,  2194, 19184,  ...,  1998,  2191,   102],
        [  101,  1996,  2449,  ...,  2007, 29461,   102],
        [  101,  5821,  2591,  ...,  2678,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [33]:
print(f"Input IDs shape: {linkedin_encodings['input_ids'].shape}")

Input IDs shape: torch.Size([104148, 256])


In [34]:
indeed_encodings = tokenize_data(indeed_job_df_clean_sw["description"], tokenizer)

In [35]:
print(indeed_encodings)

{'input_ids': tensor([[  101,  6412,  1024,  ...,  3463,  1998,   102],
        [  101,  1996,  8013,  ...,  6834,  1998,   102],
        [  101,  2055,  2566,  ...,  1013, 12191,   102],
        ...,
        [  101,  3779,  1004,  ...,  1996,  2291,   102],
        [  101,  3477,  4695,  ...,  1010,  3825,   102],
        [  101,  2597, 12654,  ...,  1997,  2326,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [36]:
print(f"Input IDs shape: {indeed_encodings['input_ids'].shape}")

Input IDs shape: torch.Size([939, 256])
